In [1]:
import pandas as pd
import numpy as np
from numpy.random import choice
import my_pickle as mp
import matplotlib.pyplot as plt

from importlib import reload

In [42]:
reload(mp)

<module 'my_pickle' from '/Users/gandalf/Documents/Galvanize/MatchingService/my_pickle.py'>

In [194]:
user_df = mp.unjson_it('data_user')
convo_df = mp.unjson_it('data_convo')

# print("Make sure we're not using ECT data")
# print(convo_df.timestamp.max())

In [197]:
user_df.hobbies

00DMWuKDFs    [Sports, Movies, TV Shows, Outdoors, Art, Soci...
00kOFrnYJu              [Gaming, Art, Movies, TV Shows, Foodie]
00xCAIRuOH    [Parties, Sports, Movies, TV Shows, Outdoors, ...
00yWCOsIvK                                          [Book Worm]
013LzOrVju    [Parties, Art, Socializing, Foodie, Running, T...
0152VWcInL                                   [Movies, TV Shows]
01DE0NCjwh           [Socializing, Art, Foodie, Yoga, TV Shows]
01Kc0ZYNae                    [Gaming, Parties, Foodie, Movies]
01VtzD3st0                                  [Movies, Book Worm]
01boaLS3Ef    [Socializing, Sports, Foodie, Gaming, Movies, ...
01iDVeaXl1    [Parties, Movies, Running, Foodie, Outdoors, A...
01mfWG4vUm      [Gaming, Socializing, Movies, TV Shows, Foodie]
01qEx4cUTH                                             [Foodie]
021QQEg1bF    [Outdoors, Socializing, Foodie, Sports, TV Sho...
025AJiLvBy    [Parties, Movies, Running, Foodie, Outdoors, A...
02GDyQPLII              [Art, Foodie, Ga

In [168]:
user_df = mp.reinstate_date(user_df,['activeAt', 'available', 'birthday','created','updated'])

# GET USER DATA FOR SENDER AND RECEIVER

In [198]:
old_names = user_df.columns
new_names = {old:old+'_sender' for old in old_names}
convo_df = convo_df.join(user_df, on='uid_sender').join(user_df, on='uid_receiver', rsuffix="_receiver").rename(index=str, columns = new_names)
convo_df.head()

,conv_id,len_receiver,len_sender,mid_receiver,mid_sender,response,timestamp,uid_receiver,uid_sender,I_count_sender,...,picture_receiver,question_count_receiver,question_ratio_receiver,sentence_count_receiver,sentence_ratio_receiver,smokingOk_receiver,term_receiver,type_receiver,updated_receiver,work_receiver
100,1Jqbth3nqH0EWNOmyQmW,NaN,17.0,None,-Kt7F-lKnoanl54lMnUO,False,2017-09-03 14:58:30.759,0EWNOmyQmW,1Jqbth3nqH,3,...,True,0,0.0,6,0.015789,0,NaN,None,1515766221838,Insightly
10002,caEwxVJXPTGyR6cCT5K8,NaN,9.0,None,-KhP4gs1cj3-MfN7aZkb,False,2017-04-10 22:58:15.738,caEwxVJXPT,GyR6cCT5K8,0,...,True,0,0.0,10,0.022272,1,NaN,None,1515753600228,Shamblott Family Dentistry
10003,caZHBg3cEbGhtafu1Ef5,NaN,36.0,None,-KnNpug-1BJr75KDw7lH,False,2017-06-24 06:43:51.751,caZHBg3cEb,Ghtafu1Ef5,0,...,True,0,0.0,1,0.005319,1,12.0,private,1515761299532,None
10008,ccY3lAJp8URACdaEvb0s,NaN,24.0,None,-KuB6nmPi4W96oF6J_pc,False,2017-09-16 19:16:51.128,ccY3lAJp8U,RACdaEvb0s,8,...,True,0,0.0,0,0.000000,1,24.0,private,1515774394803,The Black Cat
10009,ccYEEH0Ug7FH2hR293u1,26.0,16.0,-KindiRhJWOmeQ0Nt43Q,-KimFk44pXMwVVvWqAVl,True,2017-04-28 01:53:07.603,FH2hR293u1,ccYEEH0Ug7,0,...,True,0,0.0,5,0.020000,0,12.0,private,1515755919313,None


In [201]:
def roommate_rules(roommates):
    if roommates > 4: return 3
    elif roommates > 1: return 2
    elif roommates > 0: return 1
    else: return 0

def get_rent_range(row):
    max1 = row.maxCost_sender
    max2 = row.maxCost_receiver
    min1 = row.minCost_sender
    min2 = row.minCost_receiver
    if max1*max2*min1*min2 > 0:
        upper = min(max1,max2)
        lower = max(min1,min2)
        if upper-lower > 0: return upper-lower
        else: return 0
    else: return 175
    
def my_distance(row):
    a = row.location_receiver
    b = row.location_sender
    try: 
        one = b[0]-a[0]
        two = b[1]-a[1]
        return (one**2+two**2)**(.5)
    except:
        return None

def feature_time(df):
    df['age_dif'] = abs(df.age_sender-df.age_receiver)
    df['same_gender'] = df.gender_sender==df.gender_receiver
    df['same_relate'] = df.inRelationship_sender == df.inRelationship_receiver
    df['same_clean'] = df.isClean_sender == df.isClean_receiver
    df['same_night'] = df.isNight_sender == df.isNight_receiver
    df['same_student'] = df.isStudent_sender == df.isStudent_receiver
    df['same_student'] = df.isStudent_sender == df.isStudent_receiver
    df['same_smoking'] = df.smokingOk_sender == df.smokingOk_receiver
    df['same_type'] = df.type_sender == df.type_receiver
    df['same_term'] = df.term_sender == df.term_receiver
    df['same_work'] = df.work_sender == df.work_receiver
    df['same_city'] = df.hometownCity_sender == df.hometownCity_receiver
    df['same_state'] = df.hometownState_sender == df.hometownState_receiver
    df['same_country'] = df.hometownCountry_sender == df.hometownCountry_receiver
    df['same_college'] = df.college_sender == df.college_receiver
    df['roommate_similarity'] = abs(df.numRoommates_sender.apply(lambda x: roommate_rules(x))
                                  -df.numRoommates_receiver.apply(lambda x: roommate_rules(x)))
    df['hobbies_receiver'] = df.hobbies_receiver.apply(lambda x: set() if not x else set(x))
    df['hobbies_sender'] = df.hobbies_sender.apply(lambda x: set() if not x  else set(x))
    df['hobbies_overlap'] = df.apply(lambda x: len(x['hobbies_receiver'].intersection(x['hobbies_sender'])), axis=1)
    
    df['amenities_receiver'] = df.amenities_receiver.apply(lambda x: set() if not x else set(x))
    df['amenities_sender'] = df.amenities_sender.apply(lambda x: set() if not x else set(x))
    df['amenities_overlap'] = df.apply(lambda x: len(x['amenities_receiver'].intersection(x['amenities_sender'])), axis=1)

#     df['neighborhoods_receiver'] = df.neighborhoods_receiver.apply(lambda x: set() if not x  else set(x))
#     df['neighborhoods_sender'] = df.neighborhoods_sender.apply(lambda x: set() if not x  else set(x))
    df['neighborhoods_receiver'] = df.neighborhoods_receiver.apply(set)
    df['neighborhoods_sender'] = df.neighborhoods_sender.apply(set)
    df['neighborhoods_overlap'] = df.apply(lambda x: len(x['neighborhoods_receiver'].intersection(x['neighborhoods_sender'])), axis=1)

    df['rent_overlap'] = df.apply(get_rent_range, axis=1)

#     df['urgency_receiver'] = df.available_receiver-df.timestamp
#     df['urgency_sender'] = df.available_sender-df.timestamp 
    
    df['distance'] = df.apply(my_distance, axis=1)
    
    TF = {True: 1, False: 0, 'male':1, 'female':0, 'shared':0,'private':1,'nan':-1}
#     
    col_to_TF = ['response', 
                 'has_about_sender', 'has_about_receiver', 'gender_sender','gender_receiver',
                 'facebookId_sender', 'linkedinId_sender', 'picture_sender', 'has_room_sender',
                 'type_sender','type_receiver',
                 'facebookId_receiver', 'linkedinId_receiver', 'picture_receiver', 'has_room_receiver',
                 'same_work','same_city','same_state','same_country','same_college',
                 'same_gender','same_relate','same_clean','same_night','same_student','same_smoking','same_term','same_type']
    for col in col_to_TF:
        df[col] = df[col].map(TF)
    
    print("columns with null values: {}".format(len(df.columns[df.isnull().any()])))
    return df

def drop_columns(df):
    col_to_drop = ['conv_id','uid_sender','uid_receiver','mid_sender','mid_receiver',
                   'updated_sender','activeAt_sender',
                   'created_sender','available_sender','about_sender','birthday_sender', 'location_sender', 'work_sender',
                   'hometownCity_sender','hometownState_sender','hometownCountry_sender','college_sender','neighborhoods_sender',
                   'amenities_sender', 'hobbies_sender',
                   'updated_receiver','activeAt_receiver',
                   'created_receiver','available_receiver','about_receiver','birthday_receiver', 'location_receiver', 'work_receiver',
                   'hometownCity_receiver','hometownState_receiver','hometownCountry_receiver','college_receiver','neighborhoods_receiver',
                   'amenities_receiver', 'hobbies_receiver']
                   
    return df.drop(col_to_drop, axis=1)   

master_df = feature_time(convo_df)
df = drop_columns(master_df)
df = df.fillna(df.mean(axis=0))
df.head()

columns with null values: 42


,len_receiver,len_sender,response,timestamp,I_count_sender,I_ratio_sender,age_sender,exclaim_count_sender,exclaim_ratio_sender,facebookId_sender,...,same_city,same_state,same_country,same_college,roommate_similarity,hobbies_overlap,amenities_overlap,neighborhoods_overlap,rent_overlap,distance
100,16.316247,17.0,0,2017-09-03 14:58:30.759,3,0.008772,23,0,0.000000,1,...,0,1,1,0,1,3,0,0,175.0,0.552752
10002,16.316247,9.0,0,2017-04-10 22:58:15.738,0,0.012595,26,0,0.001850,1,...,0,0,0,0,1,3,0,0,175.0,0.068024
10003,16.316247,36.0,0,2017-06-24 06:43:51.751,0,0.000000,27,0,0.000000,0,...,0,0,0,0,0,1,3,0,350.0,9.291211
10008,16.316247,24.0,0,2017-09-16 19:16:51.128,8,0.024768,25,1,0.003096,1,...,0,0,0,0,0,7,1,4,100.0,0.092504
10009,26.000000,16.0,1,2017-04-28 01:53:07.603,0,0.012595,20,0,0.001850,0,...,0,0,0,1,1,3,2,0,50.0,0.113974


# PICKLE

In [143]:
mp.json_it(df,'data_features')
mp.json_it(master_df,'data_master')